In [1]:
import matplotlib.pyplot as plt, multiprocessing as mp, gc
import pandas as pd, numpy as np, random, pickle, os, time
from transition_util import *
from mpl_toolkits.mplot3d import Axes3D

In [2]:
x_max = 200
y_max = 200
z_max = 100

if not(os.path.isfile("transitions1_xyz_" + str(x_max) + "_" + str(y_max) + "_" + str(z_max) + "_multi_dict.pkl")):
    landing_ac_data = pd.read_csv("../../landing_samples.csv",
                                  dtype = {"id": str, "lat": np.float16,
                                           "lon": np.float16, "ts": np.int32, "altitude": np.int32,
                                           "climb_rate": np.int16, "azimuth": np.int16,
                                           "ground_speed": np.int16, "jfk_landing_flag": bool,
                                           "source": str, "destination": str, "ground_flag": bool,
                                           "x": np.int8, "y": np.int8, "z": np.int8,
                                           "id_start_ts": np.int32, "id_end_ts": np.int32},
                                  usecols = ["id", "lat", "lon", "ts", "altitude", "climb_rate",
                                             "azimuth", "ground_speed", "jfk_landing_flag",
                                             "source", "destination", "ground_flag", "x", "y",
                                             "z", "id_start_ts", "id_end_ts"])
    landing_ac_data = append_xyz(landing_ac_data, x_max, y_max, z_max)
    landing_ac_data = landing_ac_data.sort_values(['id', 'ts']).reset_index(drop = True)
    last_df = landing_ac_data[['id', 'ts']].groupby(['id']).last().reset_index(drop = False)
    last_df.columns = ['id', 'id_end_ts']
    landing_ac_data = landing_ac_data.drop(['id_start_ts', 'id_end_ts'], axis = 1)
    landing_ac_data = landing_ac_data.merge(last_df)

In [3]:
gamma = 0.99 ** (1/1200) # Value becomes R_max * 0.99 if aircraft lands after 20 mins
R_landing = 100
R_crash = -100
R_conflict = -5
if not(os.path.isfile("transitions1_xyz_" + str(x_max) + "_" + str(y_max) + "_" + str(z_max) + "_multi_dict.pkl")):
    get_transitions_values_times(landing_ac_data)

transitions1_xyz = pickle.load(open("transitions1_xyz_" + str(x_max) + "_" + str(y_max) + "_" + str(z_max) + "_multi_dict.pkl", "rb"))
delta_ts1 = pickle.load(open("delta_ts1_" + str(x_max) + "_" + str(y_max) + "_" + str(z_max) + "_multi_dict.pkl", "rb"))
landed_xyz = pickle.load(open("landed_xyz.pkl", "rb"))
time_to_land1_xyz = pickle.load(open("time_to_land1_" + str(x_max) + "_" + str(y_max) + "_" + str(z_max) + "_multi_dict.pkl", "rb"))
landing_value1_xyz = pickle.load(open("landing_value1_" + str(x_max) + "_" + str(y_max) + "_" + str(z_max) + "_multi_dict.pkl", "rb"))

# Non-standard aircraft

- Missed approaches: 06A11F_1, 0D0683_0, 4952A2_7, 4B187B_20, 71BE34_0, A01FEE_39, A3026F_7,
- A9F2C6_0, AAF87F_15, AB8F0D_10, ACA79D_0
- Probable jitter: 400F0B_3, 485789_2, A3D462_69, A8A785_64, AB0485_3, ABE930_6, ACB878_2, ADC934_16
- Other: 

In [4]:
# df1 = flight_data[flight_data['jfk_landing_flag']]
# # df1 = df1.sort_values(['id', 'ts']).reset_index(drop = True)
# df1 = df1.groupby(['id']).apply(lambda x: (x['destination'] == "JFK").mean()).reset_index(drop = False)
# df1 = df1[['id']][df1[0] == 1.0]
# landing_ac_data = df1.merge(flight_data)
landing_ac_data = pd.read_csv("../../landing_samples.csv",
                              dtype = {"date": str, "id": str, "lat": np.float16,
                                       "lon": np.float16, "ts": np.int32, "altitude": np.int32,
                                       "climb_rate": np.int16, "azimuth": np.int16,
                                       "ground_speed": np.int16, "jfk_landing_flag": bool,
                                       "source": str, "destination": str, "ground_flag": bool,
                                       "x": np.int8, "y": np.int8, "z": np.int8,
                                       "id_start_ts": np.int32, "id_end_ts": np.int32},
                              usecols = ["date", "id", "lat", "lon", "ts", "altitude", "climb_rate",
                                         "azimuth", "ground_speed", "jfk_landing_flag",
                                         "source", "destination", "ground_flag", "x", "y",
                                         "z", "id_start_ts", "id_end_ts"])
landing_ac_data = append_xyz(landing_ac_data, x_max, y_max, z_max)
landing_ac_data.sort_values(['id', 'ts']).reset_index(drop = True)
# landing_ac_data.drop(['id_start_ts', 'id_end_ts'], axis = 1)
first_df = landing_ac_data[['id', 'ts']].groupby(['id']).first().reset_index(drop = False)
last_df = landing_ac_data[['id', 'ts']].groupby(['id']).last().reset_index(drop = False)
first_df.columns = ['id', 'id_start_ts']
last_df.columns = ['id', 'id_end_ts']
landing_ac_data = landing_ac_data.merge(first_df)
landing_ac_data = landing_ac_data.merge(last_df)
landing_ac_data = landing_ac_data.sort_values(['id', 'ts']).reset_index(drop = True)
landing_ac_data.head()

# Retaining only IDs that have successfully landed at JFK (small range)
last_df = landing_ac_data.groupby(['id']).last().reset_index(drop = False)
last_df['coords'] = last_df.apply(lambda x: (x['x'], x['y'], x['z']), axis = 1)
last_df = last_df[last_df['coords'].apply(lambda x: x[2] <= 3)]
print(last_df['coords'].value_counts())
landing_ac_data = landing_ac_data.merge(last_df[['id']])
landing_ac_data = landing_ac_data.sort_values(['id', 'ts']).reset_index(drop = True)

(85, 113, 0)    5097
(85, 114, 0)    1799
(84, 113, 0)     325
(84, 114, 0)     125
(83, 112, 0)      25
(85, 112, 0)      11
(87, 112, 0)       5
(84, 112, 0)       4
(87, 114, 0)       2
(83, 113, 0)       2
(83, 116, 0)       1
Name: coords, dtype: int64


In [5]:
airport_coords = last_df['coords'].value_counts().index[0]

# Simulating all landings from a timestamp

In [6]:
max_time = 3600
uniq_ts = landing_ac_data['ts'].unique()
i = random.randint(a = 0, b = len(uniq_ts) - 1)
chosen_ts = uniq_ts[i]
chosen_dt = landing_ac_data['date'][landing_ac_data['ts'] == chosen_ts].iloc[0]
print(chosen_dt)
flight_data = pd.read_csv("../../" + chosen_dt + ".csv", header = None)
tmp_dat = pd.read_csv("../../samples_with_jfk_landing_flag.csv", nrows = 1)
flight_data.columns = tmp_dat.columns

2019-06-16


In [14]:
max_ts = chosen_ts + max_time
start_ts = ts1 = chosen_ts
flight_data1 = flight_data[~flight_data['jfk_landing_flag']]
landing_ac_data = landing_ac_data[(landing_ac_data['date'] == chosen_dt) & (landing_ac_data['id_end_ts'] >= chosen_ts)]
start_time = time.time()
gc.collect()
# cores = mp.cpu_count() - 1
conflict_xyz = []
total_landed = 0

def check_collision(pos, reference, current_included = True):
    return ((pos == reference).sum() - current_included)

while ts1 <= max_ts:
    print(ts1)
    last_known_other_df = flight_data1[(flight_data1['id_start_ts'] <= ts1) & (flight_data1['id_end_ts'] >= ts1)]
    last_known_other_df = last_known_other_df.sort_values(['id', 'ts']).reset_index(drop = True)
    last_known_other_df = last_known_other_df[last_known_other_df['ts'] <= ts1].groupby(['id']).last().reset_index(drop = False)
    if ts1 == start_ts:
        last_known_landing_df = landing_ac_data[(landing_ac_data['id_start_ts'] <= ts1) & (landing_ac_data['id_end_ts'] >= ts1)]
        last_known_landing_df = last_known_landing_df[(last_known_landing_df['ts'] <= ts1) & (last_known_landing_df['ground_flag'] != 1)].groupby(['id']).last().reset_index(drop = False)
    else:
        new_landing_df = landing_ac_data[(landing_ac_data['id_start_ts'] <= ts1) & (landing_ac_data['id_end_ts'] >= ts1)]
        new_landing_df = new_landing_df[~new_landing_df['id'].isin(last_known_landing_df['id'])]
        new_landing_df = new_landing_df[(new_landing_df['ts'] <= ts1) & (new_landing_df['ground_flag'] != 1)].groupby(['id']).last().reset_index(drop = False)
        last_known_landing_df = pd.concat([last_known_landing_df[last_known_landing_df['z'] != 0], new_landing_df], axis = 0)
    last_known_landing_df["dist_from_airport"] = last_known_landing_df.apply(lambda x: get_distance_from_airport(x, airport_coords), axis = 1)
    last_known_landing_df = last_known_landing_df.sort_values(['dist_from_airport']).reset_index(drop = True)
    last_known_landing_df = append_xyz(landing_ac_data = last_known_landing_df, x_max = x_max, y_max = y_max, z_max = z_max)
    last_known_landing_df['list_grSpd_azi'] = last_known_landing_df.apply(lambda x: get_next_xyzs(x['x'], x['y'], x['z'], transitions1_xyz), axis = 1)
    last_known_landing_df = last_known_landing_df[~((last_known_landing_df['ground_flag']) | (last_known_landing_df['z'] == 0))].reset_index(drop = True)
    print(last_known_landing_df.shape[0])
    df = last_known_landing_df.iloc[0]
    df = greedy_update_df(df, airport_coords, transitions1_xyz, delta_ts1, ts1)
    last_known_landing_df.iloc[0] = df
    for i in range(1, last_known_landing_df.shape[0]):
        df1 = last_known_landing_df.iloc[i]
        df2 = df1.copy(deep = True)
        delta_ts2 = 0
#         while df2['ts'] <= df['ts']:
        df2 = greedy_update_df(df2, airport_coords, transitions1_xyz, delta_ts1, df['ts'])
        if (df2['ts'] <= df['ts']):
            df1 = df2.copy(deep = True)
        last_known_landing_df.iloc[i] = df1
    total_landed += last_known_landing_df['ground_flag'].sum()
    last_known_landing_df = last_known_landing_df[last_known_landing_df['z'] > 0].reset_index(drop = True)
    current_other_df = last_known_other_df.apply(lambda x: get_current_pos(x, df['ts']), axis = 1)
    current_other_df_xyz = current_other_df.apply(lambda x: convert_pos_to_xyz_new(x, x_max, y_max, z_max))
    landing_pos_xyz = last_known_landing_df.apply(lambda x: (x['x'], x['y'], x['z']), axis = 1)
    collisions = landing_pos_xyz.apply(lambda x: check_collision(x, current_other_df_xyz, current_included = False) + check_collision(x, landing_pos_xyz, current_included = True))
    # Checking for collision between controlled aircraft and other aircraft. Non-controlled aircraft are ignored
    collisions_total = ((collisions > 0) & (last_known_landing_df['z'] > 1)).sum()
    if collisions_total > 0:
        print("Conflict occurred! Debug here.")
        conflict_xyz.append(last_known_landing_df[(last_known_landing_df['z'] > 1) & (collisions > 0)][['x', 'y', 'z']])
    if ts1 <= df['ts']:
        ts1 = df['ts'] + 1
#     elif ts1 < df['ts']:
#         ts1 = df['ts']

print(total_landed)
end_time = time.time()
print(end_time - start_time)

1560695936
19
1560695937
19


/home/naveen/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/naveen/Desktop/Coding/IBM/notebooks/flight-dqn/transition_util.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ts'] = df['ts'] + delta_ts
/home/naveen/Desktop/Coding/IBM/notebooks/flight-dqn/transition_util.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df

1560695952
19


/home/naveen/Desktop/Coding/IBM/notebooks/flight-dqn/transition_util.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['list_grSpd_azi'] = get_next_xyzs(df['x'], df['y'], df['z'], transitions1_xyz)


1560695970
19
1560695995
19
1560696020
19
1560696045
19
1560696070
19
1560696095
19
1560696120
18
1560696145
18
1560696170
18
1560696195
18
1560696220
18
1560696245
18
1560696270
17
1560696295
17
1560696320
19
1560696345
19
1560696370
19
1560696395
19
1560696420
19
1560696445
17
1560696470
17
1560696495
18
1560696520
18
1560696545
17
1560696570
17
1560696595
18
1560696620
18
1560696645
18
Conflict occurred! Debug here.
1560696670
18
1560696695
19
1560696720
19
1560696745
19
1560696770
19
1560696795
18
1560696820
17
1560696845
18
1560696870
18
1560696895
18
1560696920
18
1560696945
18
1560696970
18
1560696995
18
1560697020
18
1560697045
18
1560697070
18
1560697095
18
1560697120
18
1560697145
18
1560697170
18
1560697195
19
1560697220
19
1560697245
20
1560697270
20
1560697295
21
1560697320
21
1560697345
22
1560697370
22
1560697395
22
1560697420
23
1560697445
23
1560697470
23
1560697495
23
1560697520
23
1560697545
23
1560697570
23
1560697595
23
1560697620
23
1560697645
23
1560697670
24
156

In [15]:
total_landed

40

In [ ]:
df

In [ ]:
# last_known_landing_df.head()

In [ ]:
def greedy_update_df(df, airport_coords, transitions1_xyz, delta_ts1, ts1):
    while df['ts'] < ts1:
        nearest_ground_speed_azimuth = get_nearest_ground_speed_azimuth(df['ground_speed'], df['azimuth'], list_grSpd_azi = df['list_grSpd_azi'])
        actions, temp_delta_ts, next_xyzs = get_next_state(df = df, transitions1_xyz = transitions1_xyz, nearest_ground_speed_azimuth = nearest_ground_speed_azimuth, delta_ts1 = delta_ts1)
        actions = pd.concat(pd.Series(list(actions.keys())).apply(pd.DataFrame).tolist(), axis = 1).T
        actions.columns = ['x', 'y', 'z']
        next_xyz = tuple(get_greedy_action(actions, airport_coords))
        next_gr_spd_azi = delta_ts1[(df['x'], df['y'], df['z'])][nearest_ground_speed_azimuth][next_xyz].keys()
        next_ts = delta_ts1[(df['x'], df['y'], df['z'])][nearest_ground_speed_azimuth][next_xyz]
        delta_times = list(next_ts.values())
        delta_times = list(itertools.chain(*delta_times))
        delta_ts = int(min(delta_times))
        gr_spd_azis = [[keys] * len(next_ts[keys]) for keys in next_ts.keys()]
        gr_spd_azi = list(itertools.chain(*gr_spd_azis))
        gr_spd_azi = gr_spd_azi[delta_times.index(delta_ts)]
        df['ts'] = df['ts'] + delta_ts
        df['ground_speed'] = gr_spd_azi[0]
        df['azimuth'] = gr_spd_azi[1]
        df['x'] = next_xyz[0]
        df['y'] = next_xyz[1]
        df['z'] = next_xyz[2]
        if df['z'] != 0:
            df['list_grSpd_azi'] = get_next_xyzs(df['x'], df['y'], df['z'], transitions1_xyz)
        else:
            break
    return df

print(ts1)
df = greedy_update_df(df, airport_coords, transitions1_xyz, delta_ts1, ts1)
print(df)

In [ ]:
df = greedy_update_df(df, airport_coords, transitions1_xyz, delta_ts1)
df

In [ ]:
# print(ts1)
# last_known_other_df = flight_data1[(flight_data1['id_start_ts'] <= ts1) & (flight_data1['id_end_ts'] >= ts1)]
# last_known_other_df = last_known_other_df.sort_values(['id', 'ts']).reset_index(drop = True)
# last_known_other_df = last_known_other_df[last_known_other_df['ts'] <= ts1].groupby(['id']).last().reset_index(drop = False)
# if ts1 == start_ts:
#     last_known_landing_df = landing_ac_data[(landing_ac_data['id_start_ts'] <= ts1) & (landing_ac_data['id_end_ts'] >= ts1)]
#     last_known_landing_df = last_known_landing_df[(last_known_landing_df['ts'] <= ts1) & (last_known_landing_df['ground_flag'] != 1)].groupby(['id']).last().reset_index(drop = False)
# else:
#     new_landing_df = landing_ac_data[(landing_ac_data['id_start_ts'] <= ts1) & (landing_ac_data['id_end_ts'] >= ts1)]
#     new_landing_df = new_landing_df[~new_landing_df['id'].isin(last_known_landing_df['id'])]
#     new_landing_df = new_landing_df[(new_landing_df['ts'] <= ts1) & (new_landing_df['ground_flag'] != 1)].groupby(['id']).last().reset_index(drop = False)
#     last_known_landing_df = pd.concat([last_known_landing_df[last_known_landing_df['z'] != 0], new_landing_df], axis = 0)
# last_known_landing_df["dist_from_airport"] = last_known_landing_df.apply(lambda x: get_distance_from_airport(x, airport_coords), axis = 1)
# last_known_landing_df = last_known_landing_df.sort_values(['dist_from_airport']).reset_index(drop = True)
# last_known_landing_df = append_xyz(landing_ac_data = last_known_landing_df, x_max = x_max, y_max = y_max, z_max = z_max)
# last_known_landing_df = last_known_landing_df[last_known_landing_df['z'] > 0].reset_index(drop = True)
# last_known_landing_df['list_grSpd_azi'] = last_known_landing_df.apply(lambda x: get_next_xyzs(x['x'], x['y'], x['z'], transitions1_xyz), axis = 1)
# df = last_known_landing_df.iloc[0]
# df = greedy_update_df(df, airport_coords, transitions1_xyz, delta_ts1)
# last_known_landing_df.iloc[0] = df
# for i in range(1, last_known_landing_df.shape[0]):
#     df1 = last_known_landing_df.iloc[i]
#     df2 = df1.copy(deep = True)
#     delta_ts2 = 0
#     while df2['ts'] <= df['ts']:
#         df2 = greedy_update_df(df2, airport_coords, transitions1_xyz, delta_ts1)
#         if (df2['ts'] <= df['ts']):
#             df1 = df2.copy(deep = True)
#     last_known_landing_df.iloc[i] = df1
# current_other_df = last_known_other_df.apply(lambda x: get_current_pos(x, df['ts']), axis = 1)
# current_other_df_xyz = current_other_df.apply(lambda x: convert_pos_to_xyz_new(x, x_max, y_max, z_max))
# landing_pos_xyz = last_known_landing_df.apply(lambda x: (x['x'], x['y'], x['z']), axis = 1)
# collisions = landing_pos_xyz.apply(lambda x: check_collision(x, current_other_df_xyz, current_included = False) + check_collision(x, landing_pos_xyz, current_included = True))
# # Checking for collision between controlled aircraft and other aircraft. Non-controlled aircraft are ignored
# collisions_total = ((collisions > 0) & (last_known_landing_df['z'] > 1)).sum()
# if collisions_total > 0:
#     print("Conflict occurred! Debug here.")
#     conflict_xyz.append(last_known_landing_df[(last_known_landing_df['z'] > 1) & (collisions > 0)][['x', 'y', 'z']])
# ts1 = df['ts']

In [ ]:
# last_known_landing_df.head()

In [ ]:
# last_known_landing_df = last_known_landing_df[last_known_landing_df['z'] > 0]
last_known_landing_df.head()

In [ ]:
# conflicts = pd.concat(conflict_xyz, axis = 0).reset_index(drop = True)
# conflicts['pos'] = conflicts.apply(lambda x: tuple(x), axis = 1)
# conflicts['pos'].value_counts()

In [ ]:
# current_other_df = last_known_other_df.apply(lambda x: get_current_pos(x, df['ts']), axis = 1)
# current_other_df_xyz = current_other_df.apply(lambda x: convert_pos_to_xyz_new(x, x_max, y_max, z_max))
# current_other_df_xyz.head()

In [ ]:
# # collisions = current_df_xyz.apply(lambda x: check_collision(x, current_df_xyz))
# collisions = current_df_xyz.apply(lambda x: check_collision(x, current_df_xyz))
# last_known_df[(last_known_df['z'] > 2) & (collisions > 0)]

In [ ]:
# last_known_landing_df.head()

In [ ]:
# current_df_xyz.head() == current_df_xyz.iloc[0]

In [ ]:
# current_df_xyz.iloc[0]

In [ ]:
# last_known_landing_df.columns

In [ ]:
# max_ts = chosen_ts + max_time
# ts1 = chosen_ts
# flight_data1 = flight_data[~flight_data['jfk_landing_flag']]
# landing_ac_data = landing_ac_data[(landing_ac_data['date'] == chosen_dt) & (landing_ac_data['id_end_ts'] >= chosen_ts)]
# start_time = time.time()
# gc.collect()
# # cores = mp.cpu_count() - 1

# def get_next_xyzs(x, y, z):
#     return (list(transitions1_xyz[(x, y, z)].keys()))

# last_known_other_df = flight_data1[(flight_data1['id_start_ts'] <= ts1) & (flight_data1['id_end_ts'] >= ts1)]
# last_known_other_df = last_known_other_df.sort_values(['id', 'ts']).reset_index(drop = True)
# last_known_other_df = last_known_other_df[last_known_other_df['ts'] <= ts1].groupby(['id']).last()
# last_known_landing_df = landing_ac_data[(landing_ac_data['id_start_ts'] <= ts1) & (landing_ac_data['id_end_ts'] >= ts1)]
# last_known_landing_df = last_known_landing_df[(last_known_landing_df['ts'] <= ts1) & (last_known_landing_df['ground_flag'] != 1)].groupby(['id']).last()
# last_known_landing_df["dist_from_airport"] = last_known_landing_df.apply(lambda x: get_distance_from_airport(x, airport_coords), axis = 1)
# last_known_landing_df = last_known_landing_df.sort_values(['dist_from_airport']).reset_index(drop = True)
# last_known_landing_df = append_xyz(landing_ac_data = last_known_landing_df, x_max = x_max, y_max = y_max, z_max = z_max)
# last_known_landing_df['list_grSpd_azi'] = last_known_landing_df.apply(lambda x: get_next_xyzs(x['x'], x['y'], x['z']), axis = 1)
# last_known_landing_df['nearest_ground_speed_azimuth'] = np.empty((last_known_landing_df.shape[0], 0)).tolist()
# # i = 0
# # df = last_known_landing_df.iloc[i]
# print(df)
# df = greedy_update_df(df, airport_coords, transitions1_xyz, delta_ts1)
# print(df)

In [ ]:
# print(df['ts'])
# print(ts1)

In [ ]:
# print(gr_spd_azi)
# print(delta_ts)

In [ ]:
# next_ts

In [ ]:
# next_gr_spd_azi

In [ ]:
# for i in range(1, last_known_landing_df.shape[0]):
#     df1 = last_known_landing_df.iloc[i]
#     delta_ts = 0
#     while (delta_ts + df1['ts']) < df['ts']:
#         nearest_ground_speed_azimuth = get_nearest_ground_speed_azimuth(df1['ground_speed'], df1['azimuth'], list_grSpd_azi = df1['list_grSpd_azi'])
#         actions, temp_delta_ts, next_xyzs = get_next_state(df = df1, transitions1_xyz = transitions1_xyz, nearest_ground_speed_azimuth = nearest_ground_speed_azimuth)
#         actions = pd.concat(pd.Series(list(actions.keys())).apply(pd.DataFrame).tolist(), axis = 1).T
#         actions.columns = ['x', 'y', 'z']
#         next_xyz = tuple(get_greedy_action(actions, airport_coords))
#         next_gr_spd_azi = delta_ts1[(df1['x'], df1['y'], df1['z'])][nearest_ground_speed_azimuth][next_xyz].keys()
#         next_ts = delta_ts1[(df1['x'], df1['y'], df1['z'])][nearest_ground_speed_azimuth][next_xyz]
#         delta_times = list(next_ts.values())
#         delta_times = list(itertools.chain(*delta_times))
#         delta_ts = int(min(delta_times))
#         if (delta_ts + df1['ts'] < df['ts']):
#             gr_spd_azis = [[keys] * len(next_ts[keys]) for keys in next_ts.keys()]
#             gr_spd_azi = list(itertools.chain(*gr_spd_azis))
#             gr_spd_azi = gr_spd_azi[delta_times.index(delta_ts)]
            
#             # Update time, position, ground speed, azimuth, next set of positions
#             df1['ts'] = df1['ts'] + delta_ts
#             df1['ground_speed'] = gr_spd_azi[0]
#             df1['azimuth'] = gr_spd_azi[1]
#             df1['x'] = next_xyz[0]
#             df1['y'] = next_xyz[1]
#             df1['z'] = next_xyz[2]
#             df1['list_grSpd_azi'] = get_next_xyzs(df1['x'], df1['y'], df1['z'])

In [ ]:
# i = 1
# df1 = last_known_landing_df.iloc[i]
# df2 = df1.copy(deep = True)
# delta_ts2 = 0
# while df2['ts'] <= df['ts']:
#     nearest_ground_speed_azimuth = get_nearest_ground_speed_azimuth(df1['ground_speed'], df1['azimuth'], list_grSpd_azi = df1['list_grSpd_azi'])
#     actions, temp_delta_ts, next_xyzs = get_next_state(df = df1, transitions1_xyz = transitions1_xyz, nearest_ground_speed_azimuth = nearest_ground_speed_azimuth)
#     actions = pd.concat(pd.Series(list(actions.keys())).apply(pd.DataFrame).tolist(), axis = 1).T
#     actions.columns = ['x', 'y', 'z']
#     next_xyz = tuple(get_greedy_action(actions, airport_coords))
#     next_gr_spd_azi = delta_ts1[(df1['x'], df1['y'], df1['z'])][nearest_ground_speed_azimuth][next_xyz].keys()
#     next_ts = delta_ts1[(df1['x'], df1['y'], df1['z'])][nearest_ground_speed_azimuth][next_xyz]
#     delta_times = list(next_ts.values())
#     delta_times = list(itertools.chain(*delta_times))
#     delta_ts2 = int(min(delta_times))
#     gr_spd_azis = [[keys] * len(next_ts[keys]) for keys in next_ts.keys()]
#     gr_spd_azi = list(itertools.chain(*gr_spd_azis))
#     gr_spd_azi = gr_spd_azi[delta_times.index(delta_ts2)]
#     # Update time, position, ground speed, azimuth, next set of positions
#     df2['ts'] = df2['ts'] + delta_ts2
#     df2['ground_speed'] = gr_spd_azi[0]
#     df2['azimuth'] = gr_spd_azi[1]
#     df2['x'] = next_xyz[0]
#     df2['y'] = next_xyz[1]
#     df2['z'] = next_xyz[2]
#     df2['list_grSpd_azi'] = get_next_xyzs(df2['x'], df2['y'], df2['z'])
#     if (df2['ts'] <= df['ts']):
#         df1 = df2.copy(deep = True)

In [ ]:
# # i = 1
# # df1 = last_known_landing_df.iloc[i]
# # delta_ts = 0
# # print(df1)
# nearest_ground_speed_azimuth = get_nearest_ground_speed_azimuth(df1['ground_speed'], df1['azimuth'], list_grSpd_azi = df1['list_grSpd_azi'])
# actions, temp_delta_ts, next_xyzs = get_next_state(df = df1, transitions1_xyz = transitions1_xyz, nearest_ground_speed_azimuth = nearest_ground_speed_azimuth)
# actions = pd.concat(pd.Series(list(actions.keys())).apply(pd.DataFrame).tolist(), axis = 1).T
# actions.columns = ['x', 'y', 'z']
# next_xyz = tuple(get_greedy_action(actions, airport_coords))
# next_gr_spd_azi = delta_ts1[(df1['x'], df1['y'], df1['z'])][nearest_ground_speed_azimuth][next_xyz].keys()
# next_ts = delta_ts1[(df1['x'], df1['y'], df1['z'])][nearest_ground_speed_azimuth][next_xyz]
# delta_times = list(next_ts.values())
# delta_times = list(itertools.chain(*delta_times))
# delta_ts = int(min(delta_times))
# if (delta_ts + df1['ts'] < df['ts']):
#     gr_spd_azis = [[keys] * len(next_ts[keys]) for keys in next_ts.keys()]
#     gr_spd_azi = list(itertools.chain(*gr_spd_azis))
#     gr_spd_azi = gr_spd_azi[delta_times.index(delta_ts)]

#     df1['ts'] = df1['ts'] + delta_ts
#     df1['ground_speed'] = gr_spd_azi[0]
#     df1['azimuth'] = gr_spd_azi[1]
#     df1['x'] = next_xyz[0]
#     df1['y'] = next_xyz[1]
#     df1['z'] = next_xyz[2]
#     df1['list_grSpd_azi'] = get_next_xyzs(df1['x'], df1['y'], df1['z'])

# # print(df1)

In [ ]:
# delta_ts

In [ ]:
#     last_known_landing_df['nearest_ground_speed_azimuth'] = np.empty((last_known_landing_df.shape[0], 0)).tolist()
#     last_known_landing_df['nearest_ground_speed_azimuth'] = last_known_landing_df.apply(lambda df: get_nearest_ground_speed_azimuth(df['ground_speed'], df['azimuth'], list_grSpd_azi = df['list_grSpd_azi']), axis = 1)
#     last_known_other_df = append_xyz(landing_ac_data = last_known_other_df, x_max = x_max, y_max = y_max, z_max = z_max)
#     for i in range(last_known_landing_df.shape[0]):
#         df = last_known_landing_df.iloc[i]
#         nearest_ground_speed_azimuth = get_nearest_ground_speed_azimuth(df['ground_speed'], df['azimuth'], list_grSpd_azi = df['list_grSpd_azi'])
#         actions, temp_delta_ts, next_xyzs = get_next_state(df = df, trainsitions1_xyz, nearest_ground_speed_azimuth)


In [ ]:
# len(landing_ac_data[landing_ac_data['date'] == chosen_dt]['id'].unique())

In [ ]:
# flight_data1['ground_flag'].sum()

In [ ]:
# print(flight_data.shape)
# print(flight_data1.shape)

# Run simulation of 1 aircraft

In [ ]:
def simulate_single_aircraft_landing(df):
    landed = False
    total_time = 0
    path = []
    while not landed:
#         try:
        path.append((df['x'], df['y'], df['z']))
        ts1 = df['ts']
        # We will not be able to know the complete duration of chosen aircraft before it actually lands
        temp_flight_data = flight_data[(flight_data['id_start_ts'] <= ts1) &
                                       (flight_data['id_end_ts'] >= ts1) &
                                       (flight_data['id'] != chosen_id)]
        last_known_df = temp_flight_data[temp_flight_data['ts'] <= ts1].reset_index(drop = True).groupby(['id']).last().reset_index(drop = False)
        last_known_df = last_known_df[(last_known_df['altitude'] > 0) | (last_known_df['climb_rate'] != 0)].reset_index(drop = True)
        current_df = last_known_df.apply(lambda x: get_current_pos(x, ts1), axis = 1)
        list_grSpd_azi = list(transitions1_xyz[(df['x'], df['y'], df['z'])].keys())
        nearest_ground_speed_azimuth = get_nearest_ground_speed_azimuth(df['ground_speed'], df['azimuth'], list_grSpd_azi = list_grSpd_azi)
        actions, temp_delta_ts, next_xyzs = get_next_state(df = df, trainsitions1_xyz, nearest_ground_speed_azimuth)
#         actions = transitions1_xyz[(df['x'], df['y'], df['z'])][nearest_ground_speed_azimuth]
#         temp_delta_ts = delta_ts1[(df['x'], df['y'], df['z'])][nearest_ground_speed_azimuth]
#         next_xyzs = []
#         for key in actions.keys():
#             total = sum(list(actions[key].values()))
#             next_xyzs.append(repeat(key, total))
#         next_xyzs = unlist(next_xyzs)
        
        # This should probably change to greedy selection
        j = random.randint(a = 0, b = len(next_xyzs) - 1)
        next_xyz = next_xyzs[j]
        next_ground_speed_azimuth = list(actions[next_xyz].keys())
        j = random.randint(a = 0, b = len(next_ground_speed_azimuth) - 1)
        next_ground_speed_azimuth = next_ground_speed_azimuth[j]
        all_delta_ts = temp_delta_ts[next_xyz][next_ground_speed_azimuth]
        
        # This should probably change to greedy selection
        j = random.randint(a = 0, b = len(all_delta_ts) - 1)
        temp_delta_ts = all_delta_ts[j]
        df['ts'] = df['ts'] + temp_delta_ts
    #     previous_state = ((df['x'], df['y'], df['z']), (df['ground_speed'], df['azimuth']))
        total_time += temp_delta_ts
        df['x'] = next_xyz[0]
        df['y'] = next_xyz[1]
        df['z'] = next_xyz[2]
        df['ground_speed'] = next_ground_speed_azimuth[0]
        df['azimuth'] = next_ground_speed_azimuth[1]
        path.append(next_xyz)
        landed = (df['x'], df['y'], df['z']) in landed_xyz
        current_xyz = current_df.apply(convert_pos_to_xyz)
        collisions = current_xyz.apply(lambda x: check_collision(x, (df['x'], df['y'], df['z'])))
        if collisions.sum() > 0:
            print("Collision cannot be avoided between " + last_known_df['id'][collisions] + " and " + df['id'] + " at " + str(ts1))
            break
#         except:
#             break
    return [path, total_time]


def simulate_single_aircraft_landing_i(i):
    return simulate_single_aircraft_landing(df)

In [ ]:
# Simulating landing of 1 aircraft
uniq_id = landing_ac_data['id'].unique()
max_landings = 1
# i = random.randint(a = 0, b = len(uniq_id) - 1)

start_time = time.time()
for i in range(10):
    chosen_id = uniq_id[i]
    # chosen_id = "A3F04A_0"
    dt = landing_ac_data['date'][landing_ac_data['id'] == chosen_id].iloc[0]
    print(dt)
    flight_data = pd.read_csv("../../" + dt + ".csv", header = None)
    tmp_dat = pd.read_csv("../../samples_with_jfk_landing_flag.csv", nrows = 1)
    flight_data.columns = tmp_dat.columns

    flight_data.sort_values(['id', 'ts']).reset_index(drop = True)
    flight_data.drop(['id_start_ts', 'id_end_ts'], axis = 1)
    first_df = flight_data[['id', 'ts']].groupby(['id']).first().reset_index(drop = False)
    last_df = flight_data[['id', 'ts']].groupby(['id']).last().reset_index(drop = False)
    first_df.columns = ['id', 'id_start_ts']
    last_df.columns = ['id', 'id_end_ts']
    flight_data = flight_data.merge(first_df)
    flight_data = flight_data.merge(last_df)
    flight_data = flight_data.sort_values(['id', 'ts']).reset_index(drop = True)
    flight_data = append_xyz(flight_data, x_max, y_max, z_max)
    df = landing_ac_data[landing_ac_data['id'] == chosen_id].sort_values(['ts']).reset_index(drop = True).iloc[0]
#     print(df)
    landings = 0
    
    cores = max_landings
    p = mp.Pool(processes = cores)
    split_dfs = np.array_split([i for i in range(cores)], cores)
    pool_results = p.map(simulate_single_aircraft_landing_i, split_dfs)
    p.close()
    p.join()
    
    times = []
    for i in range(len(pool_results)):
        res = pool_results[i]
        path = res[0]
        tim = res[1]
        times.append(tim)
        mat = np.zeros((flight_data['x'].max()+1, flight_data['y'].max()+1, flight_data['z'].max()+1))
        for coord in path:
            mat[coord[0], coord[1], coord[2]] += 1

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        pos = np.where(mat > 0)
        ax.scatter(pos[0], pos[1], pos[2], c='black')
        plt.show()
    
    plt.hist(times, bins = 20)
    plt.show()

end_time = time.time()
print(end_time - start_time)